<a href="https://colab.research.google.com/github/surabhi404/pyTigerGraph/blob/main/pyTigerGraph_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install pyTigerGraph

In [ ]:
# Setup
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [ ]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = "https://35test.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


## Define and Publish the Schema

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX Person (PRIMARY_ID id STRING, name STRING, email STRING, username STRING, created_at DATETIME) WITH primary_id_as_attribute="true"
  CREATE VERTEX Post (PRIMARY_ID id STRING, content STRING, posted_date DATETIME, deleted BOOL) WITH primary_id_as_attribute="true"
  CREATE VERTEX Hashtag (PRIMARY_ID tag STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX Message (PRIMARY_ID id STRING, subject STRING, body STRING) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE posted (From Person, To Post, post_date DATETIME) WITH REVERSE_EDGE="reverse_posted"
  CREATE DIRECTED EDGE liked (From Person, To Post, like_date DATETIME) WITH REVERSE_EDGE="reverse_liked"
  CREATE DIRECTED EDGE has_tag (From Post, To Hashtag) WITH REVERSE_EDGE="reverse_has_tag"
  CREATE DIRECTED EDGE sent_message (From Person, To Message, to_person STRING, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_message"
  CREATE DIRECTED EDGE received_message (From Message, To Person, from_person STRING, receive_date DATETIME, opened_date DATETIME) WITH REVERSE_EDGE="reverse_received_message"
''')
print(results)

Successfully created vertex types: [Person].
Successfully created vertex types: [Post].
Successfully created vertex types: [Hashtag].
Successfully created vertex types: [Message].
Successfully created edge types: [posted].
Successfully created reverse edge types: [reverse_posted].
Successfully created edge types: [liked].
Successfully created reverse edge types: [reverse_liked].
Successfully created edge types: [has_tag].
Successfully created reverse edge types: [reverse_has_tag].
Successfully created edge types: [sent_message].
Successfully created reverse edge types: [reverse_sent_message].
Successfully created edge types: [received_message].
Successfully created reverse edge types: [reverse_received_message].


## Create the Graph

In [ ]:
results = conn.gsql('CREATE GRAPH MyGraph(Person, Post, Hashtag, Message, posted, reverse_posted, liked, reverse_liked, has_tag, reverse_has_tag, sent_message, reverse_sent_message, received_message, reverse_received_message)')

In [ ]:
conn.graphname="MyGraph"
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="MyGraph", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

51d6aqmvn7r9mi7v07o4bo9neb68el0q


## Clone the Data

In [ ]:
!git clone https://github.com/DanBarkus/TigerGraph-101.git

Cloning into 'TigerGraph-101'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 1), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (7/7), done.


## Create Loading Jobs

### Posts

Let's take a look at what one of our files looks like so we can write a loading job.

In [ ]:
!head -n 2 /content/TigerGraph-101/posts.csv

id,content,posted_date,by_user,deleted,hashtag_1,hashtag_2,hashtag_3,hashtag_4
1,"Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl. Aenean lectus. Pellentesque eget nunc. Donec quis orci eget orci vehicula condimentum. Curabitur in libero ut massa volutpat convallis. Morbi odio odio, elementum eu, interdum eu, tincidunt in, leo.",2019-08-04 20:43:08,69,False,Compatible,Organized,,workforce


This one and the messages file will look like a mess in this display because they contain sentances of text as dummy data which makes them not show up too well in the csv preview. You can just reference the header through for loading jobs.

Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `id` column,
- `$1` is `content`,
- `$2` is `posted_date`
- and so on

In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_posts FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Post VALUES($0, $1, $2, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hashtag VALUES($5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hashtag VALUES($6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hashtag VALUES($7) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hashtag VALUES($8) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE posted VALUES($3, $0, $2) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $7) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $8) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')
print(results)

Using graph 'MyGraph'
The job load_posts is created.


### Likes

In [ ]:
!head -n 2 /content/TigerGraph-101/likes.csv

head: cannot open '/content/TigerGraph-101/likes.csv' for reading: No such file or directory


In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_likes FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE liked VALUES($1, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

Using graph 'MyGraph'
The job load_likes is created.


### Messages

In [ ]:
!head -n 2 /content/TigerGraph-101/messages.csv

id,body,subject,by_user,to_user,send_date,read_date
1,"Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula. Suspendisse ornare consequat lectus. In est risus, auctor sed, tristique in, tempus sit amet, sem. Fusce consequat. Nulla nisl.",Nulla facilisi.,22,62,2019-05-01 03:40:51,2019-06-10 03:40:51


In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_messages FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Message VALUES($0, $2, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE sent_message VALUES($3, $0, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE received_message VALUES($0, $4, $3, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END''')
print(results)

Using graph 'MyGraph'
The job load_messages is created.


### Users

In [ ]:
!head -n 2 /content/TigerGraph-101/users.csv

id,email,username,name,join_date
1,mgeistmann0@accuweather.com,mgeistmann0,Marvin Geistmann,2018-09-30 07:31:34


In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_people FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Person VALUES($0, $3, $1, $2, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

Using graph 'MyGraph'
The job load_people is created.


In [ ]:
results = conn.gsql('''
USE GRAPH MyGraph
SHOW JOBS
''')
pprint(results)

"Encountered \" <IDENTIFIER> \"JOBS \"\" at line 3, column 6.\nWas expecting one of:\n\"DATA_SOURCE\" ...\n\"edge\" ...\n\"graph\" ...\n\"group\" ...\n\"job\" ...\n\"loading\" ...\n\"query\" ...\n\"secret\" ...\n\"tag\" ...\n\"user\" ...\n\"vertex\" ...\n"


## Load Data

In [ ]:
# Load the posts file wiht the 'load_posts' job
posts_file = '/content/TigerGraph-101/posts.csv'
results = conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_posts')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Post",
          "validObject": 1000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:posted_date"
          ],
          "invalidAttributeLinesData": [
            "id,content,posted_date,by_user,deleted,hashtag_1,hashtag_2,hashtag_3,hashtag_4\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "Hashtag",
          "validObject": 1001,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "inco

In [ ]:
# Load the likes file wiht the 'load_likes' job
likes_file = '/content/TigerGraph-101/likes.csv'
results = conn.uploadFile(likes_file, fileTag='MyDataSource', jobName='load_likes')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 5001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "liked",
          "validObject": 5000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:like_date"
          ],
          "invalidAttributeLinesData": [
            "id,by_user,liked_post,liked_date\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [ ]:
# Load the messages file wiht the 'load_messages' job
messages_file = '/content/TigerGraph-101/messages.csv'
results = conn.uploadFile(messages_file, fileTag='MyDataSource', jobName='load_messages')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Message",
          "validObject": 1001,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "sent_message",
          "validObject": 1000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:sent_date"
          ],
          "invalidAttributeLinesData": [
            "id,body,subject,by_user,to_user,send_date,read_date\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "i

In [ ]:
# Load the people file wiht the 'load_people' job
people_file = '/content/TigerGraph-101/users.csv'
results = conn.uploadFile(people_file, fileTag='MyDataSource', jobName='load_people')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 101,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Person",
          "validObject": 100,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:created_at"
          ],
          "invalidAttributeLinesData": [
            "id,email,username,name,join_date\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


## Exploring the Graph

### Get Vertex and Edge Schema

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['Person', 'Post', 'Hashtag', 'Message']
Edges: ['posted', 'liked', 'has_tag', 'sent_message', 'received_message']


In [ ]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("Message"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("liked"))


## Counting Data

In [ ]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 100 Person vertices in the graph
There are 1001 Post vertices in the graph
There are 308 Hashtag vertices in the graph
There are 1001 Message vertices in the graph
--------------
Edge Counts
There are 1000 posted edges in the graph
There are 4869 liked edges in the graph
There are 2567 has_tag edges in the graph
There are 1000 sent_message edges in the graph
There are 1000 received_message edges in the graph


## Extracting Data

### Vertex/Edge Set Format

#### Getting a Vertex

In [ ]:
results = conn.getVerticesById("Person", "30")
pprint(results)

#### Or Multiple Vertices

In [ ]:
tdf1 = conn.getVerticesById("Post", ["200","400"])
pprint(tdf1)

#### Count Edges Connected to a Vertex

In [ ]:
results = conn.getEdgeCountFrom("Person", "38")
pprint(results)

#### Show all Edges Connected to a Vertex

In [ ]:
results = conn.getEdges("Post", "345")
pprint(results)

### As Pandas Dataframe
Supports all of the above in native Pandas Dataframe format.

#### All Vertices of one Type

In [ ]:
df1 = conn.getVertexDataframe("Hashtag")
print(df1)

#### One or More Vertex

In [ ]:
df2 = conn.getVertexDataframeById("Post", ["45"])
print(df2)

#### Convert Vertex/Edge Set to Dataframe
We'll use the results from the 'Or Multiple Vertices' cell. 

In [ ]:
df3 = conn.vertexSetToDataFrame(tdf1)
print(df3)

#### Get Edges

In [ ]:
df4 = conn.getEdgesDataframe("Post", "344", limit=3)
print(df4)

## Path Finding
Find paths between vertices.

Supported are:
- shortestPath - one shortest path between vertices
- allPaths - all paths within the specified edge limit

In [ ]:
results = conn.shortestPath([("Person", "50")], [("Person", "45")])
pprint(results)

## Queries

### All Hashtags from all Posts from Input User

In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  CREATE QUERY hashtags_from_person(VERTEX<Person> inPer) FOR GRAPH MyGraph SYNTAX v2 { 
  
  person = {inPer};

  tag = SELECT t FROM person:pe - (posted>) - Post:p - (has_tag>) - Hashtag:t;

  // Below line does the same thing and looks cleaner, intermediate nodes can be ignored if not being referenced
  // tag = SELECT t FROM person:pe - (posted>.has_tag>) - Hashtag:t;

  PRINT tag;
  }
  ''')
pprint(results)

"Using graph 'MyGraph'\nThe query hashtags_from_person has been added!"


### Posts that Users who Liked the Source Post Also Liked

In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  CREATE QUERY most_common_mutual_liked_post(VERTEX<Post> inPost, INT maxReturn) FOR GRAPH MyGraph SYNTAX v2 {
  TYPEDEF tuple<STRING post, INT likes> frequency;
  
  // Find the posts most liked by the group of people who liked the input post
  HeapAccum<frequency>(maxReturn, likes DESC) @@topTestResults;
  SumAccum<INT> @likes;
  
  post = {inPost};
  
  ml = SELECT op FROM post - (<liked) - Person - (liked>) - Post:op
  ACCUM
    op.@likes += 1
  POST-ACCUM
    @@topTestResults += frequency(op.id, op.@likes);
  
  PRINT @@topTestResults;
}
  ''')
pprint(results)

"Using graph 'MyGraph'\nThe query most_common_mutual_liked_post has been added!"


### Similarity Algo

Jaccard similarity-esque algorithm that will be referenced as a sub-query.

In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  CREATE QUERY similarity(Set<STRING> A, Set<STRING> B) FOR GRAPH MyGraph RETURNS (FLOAT){ 
	SetAccum<STRING> @@inter, @@uni;
	FLOAT similarity;
	
	IF A.size() != 0 AND B.size() !=0 THEN
	  @@inter = A INTERSECT B;
	  @@uni = A UNION B;
	
	  IF @@inter.size() == 0 THEN
	    similarity = 0;
	  ELSE 
	    similarity = @@inter.size()*1.0/@@uni.size();
	    END;
	ELSE
	  similarity = 0;
	  END;
	
	PRINT similarity;
	RETURN similarity;
}
  ''')
pprint(results)

"Using graph 'MyGraph'\nThe query similarity has been added!"


### Find Users Who Like Similar Hashtags to what Input User Posts

In [ ]:
results = conn.gsql('''
  USE GRAPH MyGraph
  CREATE QUERY people_with_similar_tags(VERTEX<Person> inPer, INT maxReturn) FOR GRAPH MyGraph SYNTAX v2{ 
  TYPEDEF tuple<STRING person, FLOAT tag> simTags;
  
  HeapAccum<simTags>(maxReturn, tag DESC) @@topTagResults;
  SetAccum<STRING> @tags;
  SetAccum<STRING> @@inTags;
  person = {inPer};
  people = {Person.*};
  
  // Get the hashtags of our input person's posts
  ourTags = SELECT t FROM person - (posted>.has_tag>) - Hashtag:t
  ACCUM
    @@inTags += t.tag;
  
  // Get the hashtags of all posts all people have liked
  simPeople = SELECT p FROM people:p - (liked>.has_tag>) - Hashtag:t
  ACCUM
    p.@tags += t.tag
  POST-ACCUM
  // Compare the group of hashtags from our input person to that of the person we're currently checking
    @@topTagResults += simTags(p.id,similarity(@@inTags,p.@tags));
  
  PRINT @@topTagResults;
  }
  ''')
pprint(results)

"Using graph 'MyGraph'\nThe query people_with_similar_tags has been added!"


## Installing Queries

Installing the queries will take about 6 minutes total.

In [ ]:
conn.gsql('''
  USE GRAPH MyGraph
  INSTALL QUERY hashtags_from_person
  INSTALL QUERY most_common_mutual_liked_post
  INSTALL QUERY similarity
  INSTALL QUERY people_with_similar_tags
''')

'Using graph \'MyGraph\'\nStart installing queries, about 1 minute ...\nsimilarity query: curl -X GET \'https://127.0.0.1:9000/query/MyGraph/similarity?A=VALUE&B=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\n'

## Run Queries

### Hashtags a Person has Posted

Returned as a Vertex Set, note that the additional vertex information is returned as well as the name.

In [ ]:
results = conn.runInstalledQuery("hashtags_from_person", params={"inPer": "50"})
pprint(results)

### Posts Most Liked by the People who Liked the Input Post

- `inPost` is the input post, currently set to post id `500`. Feel free to change that and see how other posts relate.
- `maxReturn` is the max number of like posts to return. Currently set to `10`

In [ ]:
results = conn.runInstalledQuery("most_common_mutual_liked_post", params={"inPost": "500","maxReturn": "10"})
pprint(results)

### People who Like Posts with Similar Hashtags to what Input User Posts

- `inPer` is the input person who you are finding similar people to.
- `maxReturn` is the max number of similar people to return.

In [ ]:
results = conn.runInstalledQuery("people_with_similar_tags", params={"inPer": "50","maxReturn": "10"})
pprint(results)

## Clear the Whole Graph
DANGER ZONE

In [ ]:
conn.gsql('''
USE GLOBAL
DROP ALL
''')

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nTry to abort all loading jobs on graph MyGraph, it may take a while ...\n[ABORT_SUCCESS] No active Loading Job to abort.\nResetting GPE...\nSuccessfully reset GPE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 4.580 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.091 seconds\nEverything is dropped.'